In [1]:
from pathlib import Path
import pandas as pd
import pickle
import xlwings as xw
IDX = pd.IndexSlice
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

ModuleNotFoundError: No module named 'xlwings'

In [ ]:
eev_file = Path("C:/Users/WGO/Desktop/statsview_ipynb/files/energiebilanzen/pickles/eev_df.p")

files\energiebilanzen\pickles\eev_df.p

In [ ]:
_provinces = [
"Bgd",
"Ktn",
"Noe",
"Ooe",
"Sbg",
"Stk",
"Tir",
"Vbg",
"Wie",
]

_years=list(range(2000,2019,1))

c = {
    "mwh_2_gwh": 0.001,
    "gwh_2_tj": (1 / 0.27778),
    "tj_2_pj": 0.001,
    "gwh_2_mwh": 1000,
    "tj_2_gwh": 0.27778,    
    "tj_2_twh": 0.27778 / 1000,
    "pj_2_tj": 1000,
}

conv = c["tj_2_twh"]

In [4]:
eev_df = pickle.load(
    open(eev_file, "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\WGO\\Desktop\\statsview_ipynb\\files\\energiebilanzen\\pickles\\eev_df.p'

In [5]:
_field = "Energetischer Endverbrauch"

_energy_sources = ["Gesamtenergiebilanz"]
_years=list(range(2000,2019,1))
dfs = []

for province in _provinces:
    eev_data = eev_df.loc[
        IDX[_field, "Gesamt", "Gesamt", "Gesamt", "Gesamt"], IDX[province,_energy_sources , _years]
    ].round(0)
    name = "-".join([x for x in eev_data.name if x != "Gesamt"])
    eev_data.name = province
    eev_data = eev_data.droplevel([0,1], axis=0)
    dfs.append(eev_data)
    
df_tj = pd.concat(dfs,axis=1)
df_gwh = (df_tj * c["tj_2_gwh"]).round(0)
df_twh = (df_tj * c["tj_2_twh"]).round(0)

In [6]:
# wb.save(Path('data.xlsx'))
results_file = "results.xlsx"
wb = xw.Book(results_file)

try:
    wb.sheets.add(sheet_name)
    ws = wb.sheets[sheet_name]
except:
    ws = wb.sheets[sheet_name]

ws.clear_contents()

ws.range("K1").value = f"Titel: {name}"
ws.range('K1').api.Font.Bold = True
# ws.range("K2").value = f'Quelle: {results["source"]}'
# next_row_number = ws.range("K1").end('down').row

ws.range("K4").options(
    expand='table').value = df_tj
ws.range("K4").value = "TJ"

# next_column_number = ws.range("K5").end('right').get_address(0, 0)[0]
next_row_name_2 = "W4"

for u in energy_units:

    ws.range(next_row_name_2).options(
        expand='table').value = biv_gesamt[u].iloc[-1, :].to_frame().T

    ws.range(next_row_name_2).value = u

    next_row_number_2 = ws.range(next_row_name_2).end('down').row + 2
    next_row_name_2 = "W" + str(next_row_number_2)

____________________________________________________________________________________________________
Terajoule
          Bgd      Ktn       Noe       Ooe      Sbg       Stk      Tir      Vbg       Wie
YEAR                                                                                     
2000  28078.0  68769.0  203129.0  196868.0  55824.0  159328.0  69245.0  34266.0  120571.0
2001  30315.0  74842.0  220398.0  206954.0  60006.0  165626.0  73362.0  36081.0  128752.0
2002  30811.0  74754.0  218514.0  206782.0  62474.0  165777.0  75451.0  36636.0  132786.0
2003  31947.0  80365.0  224773.0  224670.0  65615.0  171594.0  80138.0  38259.0  140241.0
2004  32603.0  80173.0  231810.0  221785.0  67133.0  178596.0  81602.0  39066.0  141124.0
2005  33028.0  83136.0  238784.0  227310.0  71680.0  182228.0  85778.0  40315.0  141892.0
2006  33974.0  86072.0  239124.0  227556.0  71248.0  184296.0  85387.0  40201.0  139583.0
2007  33477.0  84769.0  235765.0  224347.0  70256.0  183332.0  84267.0  39636.0

In [2]:
wb = xw.Book(out)
for et in _energy_sources:
    try:
        wb.sheets.add(et)
    except:
        pass
    ws = wb.sheets[et]
    ws.range("A1").value = et + "-" + _field
    ws.range('A1').api.Font.Bold = True
    ws.range("A2").value = "TWh"
    ws.range("A3").value = df_tj
#     ws.range("B3:B22").color = "green"
    
    chart = ws.charts.add()
    chart.width = 500
    chart.height = 300

    chart.set_source_data(ws.range('B3').expand())
    chart.chart_type = 'column_stacked'    

    chart.api[1].SetElement(2)  # Place chart title at the top
    chart.api[1].SeriesCollection(1).XValues = _years

    for enum,x in enumerate(_provinces):
        chart.api[1].SeriesCollection(enum+1).Format.Fill.ForeColor.RGB = 22     
#         chart.api[1].SeriesCollection(enum+1).HasDataLabels = True

    chart.api[1].Axes(1).HasTitle = True # Change text of the chart title    
    chart.api[1].Axes(2).HasTitle = True # Change text of the chart title
    chart.api[1].Axes(2).AxisTitle.Text = "TJ" # Change text of the chart title
    chart.api[1].Axes(1).AxisTitle.Text =  "Years" # Change text of the chart title
    chart.api[1].Axes(1).AxisTitle.Font.Bold = False # Change text of the chart title
    chart.api[1].Axes(2).AxisTitle.Font.Bold = False # Change text of the chart title    
    chart.api[1].Axes(1).AxisTitle.Font.Bold = False # Change text of the chart title
    chart.api[1].Axes(1).TickLabels.Orientation = 90 
    chart.api[1].SetElement(2)  # Place chart title at the top
    chart.api[1].ChartTitle.Text = 'Feb sales' 
    
#     chart.api[1].SeriesCollection(1).Interior.Color = "green"

#     chart.api[1].SeriesCollection(2).Format.Line.ForeColor.RGB = RGB(255, 255, 0)

NameError: name 'xw' is not defined

In [20]:
xw.utils.int_to_rgb((0))

(0, 0, 0)

CHART TYPES

chart.chart_type = 'column_stacked'
chart.chart_type = 'bar_stacked'
chart.chart_type = 'bar_grouped'

       .ChartType = xlXYScatter

        .SeriesCollection.NewSeries
        'Chart properties
        'Title Activated, Text, and properties.
        .HasTitle = True
        .ChartTitle.Characters.Text = " Measured data vs Calculated Data" ' Chart Title
        .ChartTitle.Font.Size = 18
        .ChartTitle.Font.Bold = True
        .ChartTitle.Font.Underline = True
        .ChartTitle.Font.ColorIndex = 1
        'X axis title activate, Text, and properties
        .Axes(xlCategory, xlPrimary).HasTitle = True
        .Axes(xlCategory, xlPrimary).AxisTitle.Characters.Text = "x-Axis"
        .Axes(xlCategory, xlPrimary).AxisTitle.Font.Size = 16
        .Axes(xlCategory, xlPrimary).AxisTitle.Font.Bold = True
        .Axes(xlCategory, xlPrimary).AxisTitle.Font.Underline = True
        .Axes(xlCategory, xlPrimary).AxisTitle.Font.ColorIndex = 1
        'y axis title activate, Text, and properties
        .Axes(xlValue, xlPrimary).HasTitle = True
        .Axes(xlValue, xlPrimary).AxisTitle.Characters.Text = "Y-Axis"
        .Axes(xlCategory, xlPrimary).AxisTitle.Font.Size = 16
        .Axes(xlCategory, xlPrimary).AxisTitle.Font.Bold = True
        .Axes(xlCategory, xlPrimary).AxisTitle.Font.Underline = False
        .Axes(xlCategory, xlPrimary).AxisTitle.Font.ColorIndex = 1

        'first Series properties
            .SeriesCollection(1).Name = "=""Values""" 'name of the series
            .SeriesCollection(1).XValues = "=Sheet1!$d$4:$d$7" ' X values range
            .SeriesCollection(1).Values = "=Sheet1!$c$4:$c$7" ' Y values range
            .SeriesCollection(1).MarkerStyle = 1 'Marker's style 1= Square
            .SeriesCollection(1).MarkerSize = 7 'Marker's size
            .SeriesCollection(1).MarkerBackgroundColorIndex = 2 'Marker's colorfill 2= white
            .SeriesCollection(1).MarkerForegroundColorIndex = 1 ' Marker's border color 1=black
        .SeriesCollection.NewSeries
        .SeriesCollection(2).XValues = "=Sheet1!$e$4:$e$5"
        .SeriesCollection(2).Values = "=Sheet1!$f$4:$f$5"
        .SeriesCollection(2).Border.LineStyle = xlSolid
        .SeriesCollection(2).Border.ColorIndex = 1
        .SeriesCollection(2).MarkerBackgroundColorIndex = 2
        .SeriesCollection(2).MarkerForegroundColorIndex = 2

    End With
End Sub

NEA

In [26]:
nea_df = pickle.load(
    open(nea_file, "rb"))

In [ ]:
nea_df.loc[IDX[:], IDX["Ktn", "Gesamt (ohne E1 - E7)", "Dampferzeugung", 2000]]


RES

In [ ]:
res_data = res_data.loc[
    IDX[:, :, :, :, :], IDX["Wie", 1990]
]

In [ ]:
res_df = pickle.load(
    open(res_file, "rb"))